In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12873193012529033824
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 14848180546737486547
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]Metal device set to: Apple M1 Pro



2023-02-15 23:10:27.918681: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-15 23:10:27.918907: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [1]:
import sys
import os
import nltk
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

In [ ]:
# Define Activation Function for ReLU
def ReLU(z):
    return np.maximum(0,z)

# Derivative of ReLU
def derv_ReLU(z):
    res = (z > 0).astype(float)
    return res

# Define Activation Function for Sigmoid
def Sigmoid(z):
    return 1/(1 + np.exp(-z))

#Derivative of Sigmoid
def derv_Sigmoid(z):
    return Sigmoid(z)*(1-Sigmoid(z))

# Early Stopping criteria
def early_stopping(old_err, curr_err):
    if(old_err < curr_err):
        print("Early Stopping")
        return True
    else:
        return False
    
# Batch Normalization
def batch_normalize(x):
    mean_vec = x.mean(axis=0)
    std_vec = x.std(axis=0)
    mean_vec = mean_vec.reshape(1,mean_vec.shape[0])
    std_vec = std_vec.reshape(1,std_vec.shape[0])
    return (x - mean_vec)/std_vec

def min_max_scaleX01(data): 
    return (data - data.min(axis=0))/(data.max(axis=0) - data.min(axis=0))

def min_max_scaleY01(y):    
    return (y-1922)/(2010-1922) 

def min_max_scaleX11(data): 
    return 2*((data - data.min(axis=0))/(data.max(axis=0) - data.min(axis=0))) - 1

def min_max_scaleY11(y):    
    return 2*((y-1922)/(2010-1922)) - 1 

def min_max_scaling(train_input, train_target, dev_input, dev_target, test_input):
    train_input = min_max_scaleX01(train_input)
    train_target = min_max_scaleY01(train_target)
    dev_input = min_max_scaleX01(dev_input)
    dev_target = min_max_scaleY01(dev_target)
    test_input = min_max_scaleX01(test_input)
    return train_input, train_target, dev_input, dev_target, test_input

In [ ]:
class Net(object):
    '''
    '''

    def __init__(self, num_layers, num_units, selected_features):
        self.num_layers = num_layers
        self.num_units = num_units

        self.biases = []
        self.weights = []
        for i in range(num_layers):

            if i==0:
                # Input layer
                self.weights.append(np.random.uniform(-1, 1, size=(selected_features, self.num_units)))
            else:
                # Hidden layer
                self.weights.append(np.random.uniform(-1, 1, size=(self.num_units, self.num_units)))

            self.biases.append(np.random.uniform(-1, 1, size=(self.num_units, 1)))

        # Output layer
        self.biases.append(np.random.uniform(-1, 1, size=(1, 1)))
        self.weights.append(np.random.uniform(-1, 1, size=(self.num_units, selected_features)))

    def __call__(self, X):
        self.Z=[]
        self.A=[]   
        
        z_temp = X@self.weights[0] + (self.biases[0]).T
        z_temp = batch_normalize(z_temp)
    
        self.Z.append(z_temp)
        self.A.append(ReLU(self.Z[0]))
        
        for i in range(1,self.num_layers):
            z_temp = self.A[i-1]@self.weights[i] + (self.biases[i]).T
            z_temp = batch_normalize(z_temp)
            self.Z.append(z_temp)
            self.A.append(ReLU(self.Z[i]))
        
           
        z_temp = self.A[self.num_layers-1]@self.weights[self.num_layers] + (self.biases[self.num_layers]).T
        # z_temp = batch_normalize(z_temp)
        # self.Z.append(z_temp)
        exp_scores = np.exp(z)
        softmax_scores = exp_scores / np.sum(exp_scores, axis=0, keepdims=True)
        self.Z.append(softmax_scores)
        
        self.y_pred = self.Z[self.num_layers]
        return self.y_pred

    def backward(self, X, y, lamda):

        dz = []
        dA = []
        del_W = []
        del_b = []
        m = len(y)
        
        dz.insert(0, self.y_pred - y)
        del_W.insert(0, np.matmul(self.A[self.num_layers-1].T,dz[0]) + lamda*self.weights[self.num_layers])
        del_b.insert(0, np.matmul(np.ones((m,1)).T,dz[0]) + lamda*self.biases[self.num_layers])
        dA.insert(0, np.matmul(dz[0],self.weights[self.num_layers].T))*(1 - np.power(self.A[self.num_layers-1], 2))


        # Wt optimization for wlast yet to be written
        for i in range(self.num_layers-1, 0, -1) :

            dz.insert(0, np.multiply(dA[0],derv_ReLU(self.Z[i])))
            del_W.insert(0, np.matmul(self.A[i-1].T,dz[0]) + lamda*self.weights[i])
            
            db_cl = dz[0].sum(axis=0)
            del_b.insert(0, db_cl.reshape(db_cl.shape[0],1) + lamda*self.biases[i])
            dA.insert(0, np.matmul(dz[0],self.weights[i].T))


        dz.insert(0, np.multiply(dA[0], derv_ReLU(self.Z[0])))
        db_cl = dz[0].sum(axis=0)
        del_b.insert(0, db_cl.reshape(db_cl.shape[0],1) + lamda*self.biases[0])
        del_W.insert(0, np.matmul(X.T, dz[0]) + lamda*self.weights[0])

        return del_W, del_b

In [ ]:
class Optimizer(object):
    '''
    '''

    def __init__(self, learning_rate, optimizer):
        '''
        Create a Gradient Descent based optimizer with given
        learning rate.
        Other parameters can also be passed to create different types of
        optimizers.
        Hint: You can use the class members to track various states of the
        optimizer.
        '''
        self.learning_rate = learning_rate
        self.optimizer = optimizer
            
        
        
        self.momentum_beta = 0.9
        self.theta_weights = []
        self.theta_biases = []
        
        
        self.learning_rate_weights = learning_rate
        self.learning_rate_biases = learning_rate
        
        self.rmsprop_beta = 0.8
        self.weights_gamma = 0
        self.biases_gamma = 0
        self.weights_epsilon = 0.9
        self.biases_epsilon = 0.9

    def step(self, weights, biases, delta_weights, delta_biases):
        '''
        Parameters
        ----------
            weights: Current weights of the network.
            biases: Current biases of the network.
            delta_weights: Gradients of weights with respect to loss.
            delta_biases: Gradients of biases with respect to loss.
        '''
        if(self.optimizer == "gradient"):
            return self.gradient(weights, biases, delta_weights, delta_biases)
        elif(self.optimizer == "adam"):
            return self.adam(weights, biases, delta_weights, delta_biases)
        elif(self.optimizer == "rmsprop"):
            return self.rmsprop(weights, biases, delta_weights, delta_biases)
        elif(self.optimizer == "momentum"):
            return self.momentum(weights, biases, delta_weights, delta_biases)
        
        
        
    
    def gradient(self, weights, biases, delta_weights, delta_biases):
        for i in range(len(weights)):
            weights[i] = weights[i] - self.learning_rate_weights*delta_weights[i]
            biases[i] = biases[i] - self.learning_rate_biases*delta_biases[i]
        return weights, biases
    
    def momentum(self, weights, biases, delta_weights, delta_biases):
        '''
        Momentum Optimizer
        '''
        if(len(self.theta_weights)==0):
            self.get_theta_params(delta_weights, delta_biases)
        
        self.update_theta_params(delta_weights, delta_biases)
        
        return self.gradient(weights, biases, self.theta_weights, self.theta_biases)
       
    
    def rmsprop(self, weights, biases, delta_weights, delta_biases):
        '''
        RMSProp Optimizer
        '''       
        self.get_gamma_params(delta_weights, delta_biases)
        
        return self.gradient(weights, biases, delta_weights, delta_biases)
    
    def adam(self, weights, biases, delta_weights, delta_biases):
        '''
        ADAM Optimizer
        '''       
        if(len(self.theta_weights)==0):
            self.get_theta_params(delta_weights, delta_biases)
            
        self.update_theta_params(delta_weights, delta_biases)
        
        self.get_gamma_params(delta_weights, delta_biases)
        
        return self.gradient(weights, biases, self.theta_weights, self.theta_biases)
        
    
    def get_gamma_params(self, delta_weights, delta_biases):
        
        delta_weights_square = 0
        for i in delta_weights:
            delta_weights_square += np.sum(i**2)
            
        delta_biases_square = 0
        for i in delta_biases:
            delta_biases_square += np.sum(i**2)
            
        
        self.weights_gamma = self.rmsprop_beta*self.weights_gamma + (1-self.rmsprop_beta)*delta_weights_square
        self.biases_gamma = self.rmsprop_beta*self.biases_gamma + (1-self.rmsprop_beta)*delta_biases_square
        
        self.learning_rate_weights = self.learning_rate_weights/math.sqrt(self.weights_gamma + self.weights_epsilon)
        self.learning_rate_biases = self.learning_rate_biases/math.sqrt(self.biases_gamma + self.biases_epsilon)
        
    
    def get_theta_params(self, weights, biases):
        x = np.copy(weights)
        for i in x:
            i.fill(0)
            self.theta_weights.append(i)
            
        x = np.copy(biases)
        for i in x:
            i.fill(0)
            self.theta_biases.append(i)
            
    def update_theta_params(self, delta_weights, delta_biases):
        for i in range(len(self.theta_weights)):
            self.theta_weights[i] = self.momentum_beta*self.theta_weights[i] + (1-self.momentum_beta)*delta_weights[i]
            self.theta_biases[i] = self.momentum_beta*self.theta_biases[i] + (1-self.momentum_beta)*delta_biases[i]


In [ ]:
def cross_entropy_loss(y_pred, y_true):
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=-1))

In [ ]:
def main():

    # Hyper-parameters 
    max_epochs = 50
    learning_rate = 2e-3
    lamda = 0 # Regularization Parameter
    batch_size = 1024
    
    
    net = Net()
    optimizer = Optimizer(learning_rate, net.weights, net.biases)